In [1]:
import matplotlib.pyplot as plt

In [ ]:
19.70
22.20
22.80
25.00
22.90
21.70
19.90
21.40
21.00

Dual No CDA
Faced 22.21 std 1.45
chance 14.30

SEED_V No CDA 5 class

chance 20

In [5]:
acc_noalign = [38.47960662841797,
38.731971740722656,
35.08726119995117,
33.77593994140625,
30.231203079223633,
37.80696105957031,
31.94339370727539,
36.055946350097656]
import numpy as np
acc_mean_noalign = np.mean(acc_noalign)
acc_std_noalign = np.std(acc_noalign)
print(f'{acc_mean_noalign}±{acc_std_noalign}')

35.26403546333313±2.916800779182613


In [1]:
acc_align = [37.29155731201172,
38.833885192871094,
34.59320831298828,
36.66550064086914,
31.53864097595215,
38.607730865478516,
36.03362274169922,
36.71306228637695]
import numpy as np
acc_mean_align = np.mean(acc_align)
acc_std_align = np.std(acc_align)
print(f'{acc_mean_align}±{acc_std_align}')

36.284651041030884±2.1990436196839864


### train 9 val 1 loo:

pretrain me: 49.21±8.12

DE: 57.72±14.24


### small group finetune 4=1:3

pre model me: 46.33±3.21

finetuned: 47.88±3.20

DE：44.01±4.55

### small group finetune 8=1:7

pre model me: 42.35±2.14

finetuned: 38.92±4.35

DE：40.53±3.81



用更少样本微调（1:7）时，验证集上协方差对齐损失下降更快

### 被试内迁移(无lds)

2:8

DE: 68.04±8.02

pre model me: 45.30±4.78

finetuned: 

8:2

DE: 79.58±8.65

pre model me: 60.00±7.48